In [1]:
import spikeforest as sf

In [2]:
recordings = sf.load_spikeforest_recordings(
    'sha1://43298d72b2d0860ae45fc9b0864137a976cb76e8?hybrid-janelia-spikeforest-recordings.json'
)

In [3]:
recordings = [
    r for r in recordings
    if r.study_name == 'hybrid_static_siprobe'
]
for r in recordings:
    print(r.recording_name)

rec_16c_1200s_11
rec_16c_1200s_21
rec_16c_1200s_31
rec_16c_600s_11
rec_16c_600s_12
rec_16c_600s_21
rec_16c_600s_22
rec_16c_600s_31
rec_16c_600s_32
rec_32c_1200s_11
rec_32c_1200s_21
rec_32c_1200s_31
rec_32c_600s_11
rec_32c_600s_12
rec_32c_600s_21
rec_32c_600s_22
rec_32c_600s_31
rec_32c_600s_32
rec_64c_1200s_11
rec_64c_600s_11
rec_64c_600s_12


## Save recordings

In [4]:
import os 
import json
import numpy as np
import kachery_cloud as ka
from spikeforest.load_extractors.MdaRecordingExtractorV2.MdaRecordingExtractorV2 import readmda


def _json_serialize(x):
    if isinstance(x, np.ndarray):
        return _listify_ndarray(x)
    elif isinstance(x, np.integer):
        return int(x)
    elif isinstance(x, np.floating):
        return float(x)
    elif type(x) == dict:
        ret = dict()
        for key, val in x.items():
            ret[key] = _json_serialize(val)
        return ret
    elif type(x) == list:
        ret = []
        for i, val in enumerate(x):
            ret.append(_json_serialize(val))
        return ret
    else:
        return x

def _listify_ndarray(x):
    if x.ndim == 1:
        if np.issubdtype(x.dtype, np.integer):
            return [int(val) for val in x]
        else:
            return [float(val) for val in x]
    elif x.ndim == 2:
        ret = []
        for j in range(x.shape[1]):
            ret.append(_listify_ndarray(x[:, j]))
        return ret
    elif x.ndim == 3:
        ret = []
        for j in range(x.shape[2]):
            ret.append(_listify_ndarray(x[:, :, j]))
        return ret
    elif x.ndim == 4:
        ret = []
        for j in range(x.shape[3]):
            ret.append(_listify_ndarray(x[:, :, :, j]))
        return ret
    else:
        raise Exception('Cannot listify ndarray with {} dims.'.format(x.ndim))

In [5]:
# Save raw recordings

path = "data/hybrid_janelia/"

for R in recordings:
    recname = R.recording_name
    recfile = os.path.join(path, recname + '.json')
    obj = _json_serialize(R.recording_object)
    obj['self_reference'] = ka.store_json(
        obj, label=f'{recname}.json'
    )
    with open(recfile, 'w') as f:
        json.dump(obj, f, indent=4)
    firings_true_file = os.path.join(path, recname + '.firings_true.json')
    obj2 = R.sorting_true_object
    obj2['self_reference'] = ka.store_json(
        obj2, label=f'{recname}.firings_true.json'
    )
    with open(firings_true_file, 'w') as f:
        json.dump(obj2, f, indent=4)

    rec = R.get_recording_extractor()
    mda = readmda(rec._kwargs['raw_path'])
    np.save(os.path.join(path, recname), mda)

In [6]:
# Save true firing rates

path = "data/hybrid_janelia/"

for R in recordings:
    recname = f"{R.recording_name}.firings_true"
    print(recname)
    rec = R.get_sorting_true_extractor()
    mda = readmda(rec._kwargs['file_path'])
    np.save(os.path.join(path, recname), mda)

rec_16c_1200s_11.firings_true
rec_16c_1200s_21.firings_true
rec_16c_1200s_31.firings_true
rec_16c_600s_11.firings_true
rec_16c_600s_12.firings_true
rec_16c_600s_21.firings_true
rec_16c_600s_22.firings_true
rec_16c_600s_31.firings_true
rec_16c_600s_32.firings_true
rec_32c_1200s_11.firings_true
rec_32c_1200s_21.firings_true
rec_32c_1200s_31.firings_true
rec_32c_600s_11.firings_true
rec_32c_600s_12.firings_true
rec_32c_600s_21.firings_true
rec_32c_600s_22.firings_true
rec_32c_600s_31.firings_true
rec_32c_600s_32.firings_true
rec_64c_1200s_11.firings_true
rec_64c_600s_11.firings_true
rec_64c_600s_12.firings_true
